source: https://eshop.tesco.com.my/groceries/en-GB/

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

RETRIEVE CATEGORIES

In [2]:
start_time = time.time()
driver = webdriver.Chrome()
driver.switch_to_window(driver.current_window_handle)
driver.maximize_window()
driver.get('https://eshop.tesco.com.my/groceries/en-GB/')
# driver.find_element_by_css_selector('.menu-tree mq-l--menu-tree--open')
# GET LINKS FOR EACH-SUB CATEGORY
category = driver.find_elements_by_class_name('menu__item--superdepartment')
wanted = ['Fresh Food','Grocery','Chilled & Frozen','Drinks']
outputs = []
for stage1 in category:
    main_cat = stage1.text.split('\n')[1]
    if main_cat in wanted:
        stage1.click()
        inner1 = stage1.find_elements_by_class_name('menu__item--department')
        for stage2 in inner1[1:]:
            sub_cat = stage2.text.split('\n')[1]
            stage2.click()
            inner2 = stage2.find_elements_by_class_name('menu__item--aisle')
            for stage3 in inner2[1:]:
                aisle = stage3.text.split('\n')[1]
                url = stage3.find_element_by_tag_name('a').get_attribute('href')
                outputs.append([main_cat, sub_cat, aisle, url])
driver.implicitly_wait(1)
driver.quit()
print('*** Process Completeed ***')
print('Time used:\t',time.time()-start_time)

data = pd.DataFrame(outputs, columns=['main_cat', 'sub_cat', 'aisle', 'url'])
print('Dataframe shape:\t',data.shape)
data.to_csv('tesco_category.csv',index=False)
print(data.main_cat.unique())

*** Process Completeed ***
Time used:	 31.464118242263794
Dataframe shape:	 (194, 4)
['Fresh Food' 'Grocery' 'Chilled & Frozen' 'Drinks']


In [31]:
data = pd.read_csv('tesco_category.csv')
data['counts'] = pd.Series(item_counts)
data.counts = data.counts.astype('int')
data.to_csv('tesco_category.csv',index=False)

In [32]:
data.head()

,main_cat,sub_cat,aisle,url
0,Fresh Food,Bakery,Brown Crusty Loaves & Rolls,https://eshop.tesco.com.my/groceries/en-GB/sho...
1,Fresh Food,Bakery,Cakes,https://eshop.tesco.com.my/groceries/en-GB/sho...
2,Fresh Food,Bakery,Flat Bread,https://eshop.tesco.com.my/groceries/en-GB/sho...
3,Fresh Food,Bakery,Savouries Bun,https://eshop.tesco.com.my/groceries/en-GB/sho...
4,Fresh Food,Bakery,"Specialty, Healthy Crusty Loaves & Rolls",https://eshop.tesco.com.my/groceries/en-GB/sho...


RETRIEVE ITEMS

In [17]:
start_time = time.time()
driver = webdriver.Chrome()
driver.switch_to_window(driver.current_window_handle)
driver.maximize_window()
driver.get('https://eshop.tesco.com.my/groceries/en-GB/')

item_datas = []
item_counts = []
errors = []
for i, row in data.iloc[185:].iterrows():
#     try:
    driver.get(row.url)
    item_counts.append(int(driver.find_element_by_class_name('pagination__results-count').find_elements_by_tag_name('strong')[-1].text.split(' ')[0]))
    ### CLICK THROUGH ALL THE PAGES
    while True:   
        item_list = driver.find_elements_by_css_selector('.product-list--list-item')
        if len(item_list) == count:
            break
        driver.find_elements_by_css_selector('.prev-next')[1].click()
        driver.implicitly_wait(5)
    ### GATHER THE ITEM AND EXTRACT DATA
    for n in item_list:
        name = n.find_element_by_css_selector('.product-tile--title').get_attribute('innerText')
        price = n.find_element_by_class_name('price-per-quantity-weight').get_attribute('innerText')
        temp = price.split('/')
        try:
            promo = n.find_element_by_class_name('offer-text').get_attribute('innerText')
            promo_date = n.find_element_by_css_selector('span.dates').get_attribute('innerText')
        except:
            promo = ''
            promo_date = ''
        item_datas.append([row.main_cat, row.sub_cat, row.aisle, name, temp[0], temp[1], promo, promo_date])
#     except Exception as e:
#         errors.append([row.main_cat, row.sub_cat, row.aisle, e.__class__.__name__])
#         print([row.main_cat, row.sub_cat, row.aisle, e.__class__.__name__])
driver.implicitly_wait(1)
driver.quit()
print('*** Process Completeed ***')
print('Time used:\t',time.time()-start_time)

*** Process Completeed ***
Time used:	 360.5790169239044


Change promo to promotion, item_metric to item_vol

In [20]:
item_df = pd.DataFrame(item_datas, columns=['main_cat','sub_cat','aisle','item_name','item_price','item_metric','promo','promo_date'])
print('Dataframe shape:\t',item_df.shape)
item_df.to_csv('tesco_database.csv',index=False)
item_df.head()

(6435, 8)


,main_cat,sub_cat,aisle,item_name,item_price,item_metric,promo,promo_date
0,Fresh Food,Bakery,Brown Crusty Loaves & Rolls,Massimo Grande Sandwich Loaf With Wheat Germ 600g,RM3.63,each,NaN,NaN
1,Fresh Food,Bakery,Brown Crusty Loaves & Rolls,Massimo Fine Wholemeal Loaf 420g,RM2.90,each,NaN,NaN
2,Fresh Food,Bakery,Brown Crusty Loaves & Rolls,Gardenia Fine Wholemeal 400g,RM2.90,each,NaN,NaN
3,Fresh Food,Bakery,Brown Crusty Loaves & Rolls,Wheat Bran Bread,RM1.89,each,NaN,NaN
4,Fresh Food,Bakery,Brown Crusty Loaves & Rolls,Mighty White Wholemeal Bread 400g,RM3.50,each,NaN,NaN
